In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
import torch.nn.functional as F
import sys
import os
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.preprocessing import StandardScaler
root = "/project/wyin/jlee/ml-project/dos-prediction"
util_loc = os.path.join(root, "utils")
data_loc = os.path.join(root, "data")
sys.path.append(util_loc)
from utilities import LitNeuralNet, DosDataModule

In [2]:
models = []
same_models = []
for i in range(24):
    log_path = "logs/long-run-24"
    log_dir = os.listdir(log_path)[i]
    check_dir = os.path.join(log_path, log_dir, "version_0/checkpoints")
    check_file = os.listdir(check_dir)[0]
    check_path = os.path.join(check_dir, check_file)
    model = LitNeuralNet.load_from_checkpoint(checkpoint_path=check_path)
    model_name = log_dir.rpartition(']')[0] + ']'
    
    if model_name != "[3, 256, 512, 768, 1024, 301]":
        models.append((model_name, model))
        
    else:
        same_models.append(model)
    
    
for i in range(10):
    log_path = "logs/long-run-10"
    log_dir = os.listdir(log_path)[i]
    check_dir = os.path.join(log_path, log_dir, "version_1/checkpoints")
    check_file = os.listdir(check_dir)[0]
    check_path = os.path.join(check_dir, check_file)
    model = LitNeuralNet.load_from_checkpoint(checkpoint_path=check_path)
    model_name = log_dir.rpartition(']')[0] + ']'
    models.append((model_name, model))
    
print(len(models))
print(len(same_models))

24
10


In [30]:
dataset = np.load(f"{data_loc}/train-set.npz")
train_params = dataset['params']

dataset = np.load(f"{data_loc}/val-set.npz")
val_params_arr = dataset['params']
val_dos_arr = dataset['dos']

scaler = StandardScaler()
scaler.fit(train_params)
val_params_arr = scaler.transform(val_params_arr)

val_params_arr = torch.from_numpy(val_params_arr).float()
val_dos_arr = torch.from_numpy(val_dos_arr).float()
print(params_arr.shape)

torch.Size([13265, 3])


In [38]:
def mse(model):
    predicted = model.forward_prop(val_params_arr)
    return F.mse_loss(predicted, val_dos_arr).item()


#Test random model for sanity check
model_name, model = models[20]
print(f"{model_name} MSE: {mse(model)}")

batch_size = 1024
dos_data = DosDataModule(data_loc, batch_size)
dos_data.setup()
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience = 100)
trainer = pl.Trainer()
end_res = trainer.validate(model, dataloaders = dos_data)

[3, 256, 512, 768, 1024, 301] MSE: 5.1276829253765754e-06


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        val_loss           5.127681561134523e-06
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [41]:
sorted_models = sorted(models, key = lambda x : mse(x[1]))
for model in sorted_models:
    print(f'{model[0]}: {mse(model[1])}')

[3, 170, 340, 510, 680, 850, 1020, 301]: 4.5635088099516e-06
[3, 204, 408, 612, 816, 1020, 301]: 5.042199518356938e-06
[3, 512, 512, 512, 512, 512, 512, 512, 301]: 5.054268058302114e-06
[3, 146, 292, 438, 584, 730, 876, 1022, 301]: 5.08028506374103e-06
[3, 256, 512, 768, 1024, 301]: 5.1276829253765754e-06
[3, 128, 256, 384, 512, 640, 768, 896, 1024, 301]: 5.565386345551815e-06
[3, 512, 512, 512, 512, 512, 512, 301]: 5.6197241065092385e-06
[3, 128, 256, 512, 1024, 301]: 5.810610673506744e-06
[3, 64, 128, 256, 512, 1024, 301]: 6.176588613016065e-06
[3, 32, 64, 128, 256, 512, 1024, 301]: 7.037332579784561e-06
[3, 512, 512, 512, 512, 512, 301]: 7.051531156321289e-06
[3, 256, 512, 1024, 301]: 7.409566023852676e-06
[3, 73, 146, 219, 292, 365, 438, 511, 301]: 7.5117882261110935e-06
[3, 85, 170, 255, 340, 425, 510, 301]: 7.6142523539601825e-06
[3, 512, 512, 512, 512, 301]: 7.843245839467272e-06
[3, 64, 128, 192, 256, 320, 384, 448, 512, 301]: 7.86020973464474e-06
[3, 102, 204, 306, 408, 510, 3

In [79]:
#Ensemble with different architectures
ensemble_res = []
def ensemble(ens_size):
    total_pred = torch.zeros([13265, 301])
    total_mse = 0
    for model_name, model in sorted_models[:ens_size]:
        predicted = model.forward_prop(params_arr)
        error = mse(model)
        total_mse += error
        total_pred = torch.add(total_pred, predicted)

    avg_pred = total_pred / ens_size
    ens_mse = F.mse_loss(avg_pred, dos_arr).item()
    print(ens_mse)
    return ens_mse
    
for i in range(1, 25):
    ensemble_res.append(ensemble(i))

4.5635088099516e-06
3.7378347315097926e-06
3.4210693229397293e-06
3.3009262097039027e-06
3.1860188300925074e-06
3.1874094474915182e-06
3.1775641673448263e-06
3.163130941175041e-06
3.177038706780877e-06
3.2260377338388935e-06
3.2719708542572334e-06
3.295528586022556e-06
3.3352878290315857e-06
3.383306875548442e-06
3.4321012662985595e-06
3.4652941849344643e-06
3.5068778743152507e-06
3.5554378428059863e-06
3.6119131436862517e-06
3.687421440190519e-06
3.763792847166769e-06
3.8472426240332425e-06
3.917740741599118e-06
3.9872443267086055e-06


In [84]:
print(min(ensemble_res))
print(ensemble_res.index(min(ensemble_res)) + 1)

3.163130941175041e-06
8


In [88]:
#Ensemble with best architecture
ensemble_res = []
sorted_same_models = sorted(same_models, key = lambda x : mse(x))
def ensemble(ens_size):
    total_pred = torch.zeros([13265, 301])
    total_mse = 0
    for model in sorted_same_models[:ens_size]:
        predicted = model.forward_prop(params_arr)
        error = mse(model)
        total_mse += error
        total_pred = torch.add(total_pred, predicted)
    
    avg_pred = total_pred / ens_size
    ens_mse = F.mse_loss(avg_pred, dos_arr).item()
    print(ens_mse)
    return ens_mse

for i in range(1, 11):
    ensemble_res.append(ensemble(i))

4.932330284646014e-06
3.8558978303626645e-06
3.519755637171329e-06
3.3416256428608904e-06
3.2547968658036552e-06
3.1889212550595403e-06
3.1620181744074216e-06
3.139731461487827e-06
3.136958184768446e-06
3.168372813888709e-06


In [13]:
#Use 10 best models and average

NameError: name 'layer_sizes' is not defined